In [1]:
%load_ext autoreload
%autoreload 2

%autosave 120

import pandas as pd
import glob
import numpy as np
import os
import json
from modules.utilities import are_bugs_from_tangled, has_bug, is_same_history, parse_hash_delimited_string

Autosaving every 120 seconds


# Creating commit database

In [2]:
'''
{
    "#ProjectName#": 
    {
        "#CommmitHash#": 
        [
            {
                "File": string,
                "MethodName": string,
                "ChangeType": string
            },
            ,,,,,,
        ],
        .....
    },
    ......
}
'''

'\n{\n    "#ProjectName#": \n    {\n        "#CommmitHash#": \n        [\n            {\n                "File": string,\n                "MethodName": string,\n                "ChangeType": string\n            },\n            ,,,,,,\n        ],\n        .....\n    },\n    ......\n}\n'

In [3]:
csv_files = glob.glob("./data/Processed/*.csv")

database = {}
for csv_file in csv_files:
    df = pd.read_csv(csv_file,delimiter="\t")
    project_name = os.path.basename(csv_file).replace(".csv","")
    database[project_name] = {}

    for index, row in df.iterrows():
        json_path = os.path.join("./data/source-methods/", project_name, row["file"])
        data = json.load(open(json_path))
        for commit_hash in data["changeHistory"]:
            if commit_hash not in database[project_name].keys():
                database[project_name][commit_hash] = []
            database[project_name][commit_hash].append({
                "File": row["file"],
                "MethodName": data["functionName"],
                "ChangeType": data["changeHistoryShort"][commit_hash]
            })


In [4]:
import json

# Serialize data into file:
json.dump(database, open("./data/CommitDatabase.json", 'w'))


In [2]:

# Read data from file:
commit_db = json.load(open("./data/CommitDatabase.json"))

# Creating Risky Commit database

In [21]:
csv_files = glob.glob("./data/Processed/*.csv")

bug_database = {}
for csv_file in csv_files:
    df = pd.read_csv(csv_file,delimiter="\t")
    project_name = os.path.basename(csv_file).replace(".csv","")

    bug_database[project_name] = {}

    for _, row in df.iterrows():
        history = parse_hash_delimited_string(row["RiskyCommit"])
        history.reverse()
        
        json_path = os.path.join("./data/source-methods/", project_name, row["file"])
        data = json.load(open(json_path))

        for index, is_buggy in enumerate(history):
            if is_buggy == 1:
                bug_database[project_name][data["changeHistory"][index]] = len(commit_db[project_name][data["changeHistory"][index]])

In [22]:

json.dump(bug_database, open("./data/RiskyCommitDatabase.json", 'w'))

# Creating Buggy Commit Database

In [3]:
csv_files = glob.glob("./data/Processed/*.csv")

bug_database = {}
for csv_file in csv_files:
    df = pd.read_csv(csv_file,delimiter="\t")
    project_name = os.path.basename(csv_file).replace(".csv","")
    
    bug_database[project_name] = {}

    for _, row in df.iterrows():
        history = parse_hash_delimited_string(row["Buggycommiit"])
        history.reverse()
        
        json_path = os.path.join("./data/source-methods/", project_name, row["file"])
        data = json.load(open(json_path))

        for index, is_buggy in enumerate(history):
            if is_buggy == 1:
                bug_database[project_name][data["changeHistory"][index]] = len(commit_db[project_name][data["changeHistory"][index]])

In [4]:

json.dump(bug_database, open("./data/BuggyCommitDatabase.json", 'w'))